In [2]:
# 연습문제 : 유튜브 댓글 수집하는 크롤러

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
import openpyxl
import pyautogui
from tqdm.notebook import tqdm    #pip install tqdm

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("연습문제 : 유투브 영상의 댓글 수집하기")
print("=" *80)
query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))

f_dir = input("3.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\\py_temp\\):")
if f_dir == '' :
    f_dir = 'c:\\py_temp\\'

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+'youtube'+'-'+query_txt)
os.chdir(f_dir+s+'-'+'youtube'+'-'+query_txt)

ff_name=f_dir+s+'-'+'youtube'+'-'+query_txt+'\\'+s+'-'+'youtube'+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+'youtube'+'-'+query_txt+'\\'+s+'-'+'youtube'+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+'youtube'+'-'+query_txt+'\\'+s+'-'+'youtube'+'-'+query_txt+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( )

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get('https://www.youtube.com')
driver.maximize_window()
time.sleep(5)

element = driver.find_element(By.NAME,"search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

# 검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

if cnt < 20 :
        page_cnt = 1 
else :
        page_cnt = math.ceil(cnt/20)
        
if cnt > 20 :          
    i = 1
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(3)
        i += 1

# 검색된 유튜브 영상의 URL 을 추출합니다
time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]  # 수집할 유투브 영상의 URL 정보 저장할 변수

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):        
    item.append(i['href'])
    count += 1

    if count == cnt :
        break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %cnt)
print("\n")

# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.youtube.com/' +item[x]
    full_url.append(url)
    if url_cnt == cnt :
        break

for y in full_url :
    print(y)

time.sleep(5)
play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]
total_cnt = 0

for addr in full_url :
    driver.get(addr)
    time.sleep(5)
    pyautogui.scroll(1000)
    time.sleep(5)
    pyautogui.scroll(-500)
    time.sleep(5)
     
    print("\n")
    print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser')
    
    # 조회수 추출하기
    t_count_0 = soup2.select('#count')
    t_count_1 = t_count_0[1].get_text()
    t_count_2 = t_count_1.replace(",","")
    t_count_3 = re.search("\d+",t_count_2)
    t_count_4 = int(t_count_3.group())

    # 영상 제목 추출하기
    # 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
    bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

    t_title_1 = soup2.select('#info-contents')
    t_title_2 = t_title_1[0].find('h1').get_text()
    t_title_3 = t_title_2.translate(bmp_map).replace("\n","")

    # 댓글 수
    try :
        t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()
    except AttributeError:
        pyautogui.click(x=40,y=1490)
        pyautogui.scroll(-1000)
        time.sleep(3)
        t_view_1 = soup2.find('h1','title style-scope ytd-video-primary-info-renderer').get_text()
    else :
        t_view_2 = t_view_1.replace("," ,"")
        t_view_3 = re.search("\d+",t_view_2)
        print(t_view_1)
        t_view_4 = int(t_view_3.group())
    print('첫번째 영상의 댓글은 답글 포함해서 총 %s개입니다' %t_view_4)
# 각 동영상마다 수집할 건수를 입력하려면 아래의 5줄의 주석을 제거하고 사용하세요
#     try :
#         reple_cnt = int(input('동영상에서 추출할 댓글(답글제외)은 몇건입니까?(기본값:%s 건): ' %t_view_4))
#     except :
#         reple_cnt = t_view_4
#         print('기본값인 %s 건의 댓글을 수집하겠습니다~~' %reple_cnt)
    
    # 자동으로 각 영상마다 전체 댓글을 수집하도록 아래 줄에서 설정 했습니다.
    reple_cnt = t_view_4
    
    reple_page_cnt = math.ceil(reple_cnt/30)

    print("=" *130)
    print("%s 번째 동영상의 조회수는 %s 회 이고 수집할 댓글은 총 %s개 입니다" %(play_cnt,t_count_4,reple_cnt))  
    print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
    print("=" *130)
    print("\n")
    print("%s 번째 동영상에서 댓글 수집을 시작합니다" %play_cnt)
    if reple_cnt > 20 :
        print("댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^")
    print("\n")

    #Step 5. 화면을 스크롤해서 아래로 이동합니다
    i = 1
    while ( i <=  reple_page_cnt+1) :
        print("%s 번째 페이지의 댓글을 수집하는 중입니다~" %i)
        scroll_down(driver)
        time.sleep(3)
        i += 1

    #Step 6. 내용을 수집합니다
    html3 = driver.page_source
    soup3 = BeautifulSoup(html3, 'html.parser')
    time.sleep(2)
    
    count = 0

    reple_result = soup3.find_all('ytd-comment-thread-renderer','style-scope ytd-item-section-renderer')

    for a in tqdm( reple_result ) :
        count += 1
              
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("\n")
        f.write("------------------------------------------------------------------"+"\n")

        print("\n")
        print("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
        print("-" *70)

        # 유튜브 URL 주소
        print("1.URL 주소:",addr)
        f.write("1.유투브 URL주소: " + addr + "\n")
        url2.append(addr)

        # 댓글 작성자
        writer = a.find('span','style-scope ytd-comment-renderer').get_text().replace("\n",'').strip()
        print("2.댓글 작성자명:", writer)
        f.write("2.댓글 작성자명: " + writer + "\n")
        writer2.append(writer)

        # 댓글 작성일자
        wdate = a.find('yt-formatted-string','published-time-text above-comment style-scope ytd-comment-renderer').get_text().replace("\n","")

        print("3.댓글 작성일자:",wdate)
        f.write("3.댓글 작성일자: " + wdate + "\n")
        wdate2.append(wdate)

        # 댓글 내용
        reple1 = a.find('div','style-scope ytd-expander').get_text()

        reple2=reple1.translate(bmp_map).replace("\n","")
        print("4.댓글 내용:",reple2)
        f.write("4.댓글 내용: " + reple2 + "\n")
        reple3.append(reple2)
                            
        f.close( )
              
        if count == reple_cnt :
            break
        print('=' * 120)
        print('%s번째 영상에서 최종 %s건의 리뷰를 수집 완료했습니다~~' %(play_cnt , str(len(reple_result)) ) )
        total_cnt += 1 # 총 리뷰 건수
        print('현재까지 %s개의 영상에서 %s건의 누적 리뷰건수를 수집했습니다' %(play_cnt,total_cnt))
        print('=' * 120)
        print()
            
              
    time.sleep(2)       
    play_cnt += 1                

#step 9. 엑셀 형태로 저장하기
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)


# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=False , engine='openpyxl')

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %round(t_time,1) )
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *120)
print("1.요청된 총 %s 건 동영상 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,total_cnt))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *120)

time.sleep(3)

driver.close( )

연습문제 : 유투브 영상의 댓글 수집하기
1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): 올리브영
2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:5
3.크롤링 결과를 저장할 폴더명만 쓰세요(예:c:\py_temp\):


요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^
검색된 영상 중 5 건 동영상의 댓글을 추출하겠습니다


https://www.youtube.com//watch?v=yaE2vJCYYVI
https://www.youtube.com//watch?v=siimHal8oN0
https://www.youtube.com//watch?v=tilL_IqC16g
https://www.youtube.com//watch?v=Ax9qhZJU9lc
https://www.youtube.com//watch?v=r-o5HX4-gcA


1 번째 동영상의 정보를 수집합니다.


NameError: name 't_view_4' is not defined